In [1]:
# PROXY
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value
    
os.environ['HF_HOME'] = '/autodl-tmp/hf'

In [2]:
from datasets import load_dataset
datas = load_dataset("bigbio/biored")

/root/miniconda3/lib/python3.8/site-packages/datasets/load.py:1461: FutureWarning: The repository for bigbio/biored contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/biored
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [43]:
from pprint import pprint as print
import json

def categorize(s):
    if s == 'ChemicalEntity':
        return 'Compound'
    if s == 'DiseaseOrPhenotypicFeature':
        return 'Disease'
    if s == 'GeneOrGeneProduct':
        return 'Gene'
    if s == 'CellLine':
        return 'CellLine'

dataset = []
i = 0
while i < len(datas['train']):
    data = datas['train'][i]
    passages = data['passages']
    entities = data['entities']
    abstract = passages[1]['text'][0]
    # print(abstract)
    # print(entities)
    l = []

    for e in entities:
        s = e['semantic_type_id']
        if s == 'ChemicalEntity':
            c =  'Compound'
        elif s == 'DiseaseOrPhenotypicFeature':
            c = 'Disease'
        elif s == 'GeneOrGeneProduct':
            c = 'Gene'
        elif s == 'CellLine':
            c = 'CellLine'
        else:
            continue
        l.append({
             "entity": e['text'][0],
            "category": c
        })
    
    instruct = {
        "instruction": """To ensure the analysis is both comprehensive and accurate, it is crucial to identify and categorize biomedical entities from the text strictly according to the provided categories. Your output should only include entities that fit into the following categories: ['CellLine','ChemicalEntity','DiseaseOrPhenotypicFeature','GeneOrGeneProduct']. Any entities that do not align with these categories must be omitted.
For each identified entity, detail the following in a JSON list format:
- Entity name (it must be a biomedical entity precisely fitting into the provided categories)
- The applicable category from the provided list
Remember, adherence to the category list is non-negotiable. Continuous refinement should be based on aligning strictly with the provided categories, improving the accuracy and relevance of identified entities.
You should ensure the output fits the json format, DO NOT add comments in any forms to category if there are more than one possible answers.""",
        "input": abstract,
        "output": str(l)
    }
    dataset.append(instruct)
    i += 1

In [47]:
with open('BioRed.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=4)

In [45]:
['Gene', 'Protein', 'Compound', 'Disease', 'Symptom', 'Pathway', 'Anatomy', 'Metabolite', 'MolecularFunction', 'BiologicalProcess', 'CellularComponent']

['Gene',
 'Protein',
 'Compound',
 'Disease',
 'Symptom',
 'Pathway',
 'Anatomy',
 'Metabolite',
 'MolecularFunction',
 'BiologicalProcess',
 'CellularComponent']

In [20]:
set([j['semantic_type_id'] for i in datas["train"] for j in i['entities']])

{'CellLine',
 'ChemicalEntity',
 'DiseaseOrPhenotypicFeature',
 'GeneOrGeneProduct',
 'OrganismTaxon',
 'SequenceVariant'}